In [ ]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset
import pandas as pd
from datasets import Dataset, ClassLabel, Features, Value
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib
import datetime
from tqdm import tqdm

In [ ]:
col_list = [
    "Practical barriers to vaccination (-)",
    "Perceived barriers to accepting vaccines (-)",
    "Perceived benefits (+)",
    "Misinformation (-)",
    "Perceived Disease Risk (+)",
    "Social norms  cues to action (+)",
    "Attitude",
]

# relevant prediction
file_list = os.listdir('./content')

model = SetFitModel.from_pretrained("final_model/relevant")

def labeling(row):
    text = row['content']
    return model([text]).item()
for file in file_list:
    print(file)
    df = pd.read_csv(f'./content/{file}')
    df = df.fillna('0')
    # progress bar
    tqdm.pandas()

    df['relevant'] = df.progress_apply(labeling, axis=1)
    # save to csv
    df.to_csv(f'./results/{file[:-4]}_relevant.csv', index=False)
    print('done!')

# other prediction
file_list = [file for file in os.listdir('./results') if 'relevant' in file]
for file in file_list:
    df = pd.read_csv(f'./results/{file}')
    df = df[df['relevant'] == 1]
    for col in col_list:
        print(f'{col} start!')
        model = SetFitModel.from_pretrained(f"./final_model/{col}")
        # progress bar
        tqdm.pandas()
        
        df[col] = df.progress_apply(labeling, axis=1)
        print(f'{col} done!')
        
    # save to csv    
    df.to_csv(f'./results/all_{file}', index=False)